In [ ]:
%cd drive/MyDrive/bio/

/content/drive/MyDrive/bio


In [ ]:
!ls

dataset_loader.py  image_at_epoch_0050.png  models.py	 Test.ipynb
Experiments	   loss_functions.py	    __pycache__  UADFV


In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import time
import tensorflow as tf

from IPython import display

In [ ]:
tf.device('/device:GPU:0')
batch_size = 16
image_resize = (400, 400)
seed = 2020
data_dir = r'UADFV/development/'
test_dir = r'UADFV/evaluation/'

optimizer = tf.optimizers.Adam(0.001)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False,
   # rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    validation_split = 0.1
)

train_generator_autoencoder = data_generator.flow_from_directory(
    directory=data_dir,
    class_mode='input',
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    target_size=image_resize,
    subset="training"
)

val_generator_autoencoder = data_generator.flow_from_directory(
    directory=data_dir,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="input",
    shuffle=True,
    target_size=image_resize,
    seed=seed,
    subset="validation"
)

train_generator = data_generator.flow_from_directory(
    directory=data_dir,
    class_mode='binary',
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    target_size=image_resize,
    subset="training"
)

val_generator = data_generator.flow_from_directory(
    directory=data_dir,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    target_size=image_resize,
    seed=seed,
    subset="validation"
)

test_generator = data_generator.flow_from_directory(
    directory=test_dir,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    target_size=image_resize,
    seed=seed,
)

Found 684 images belonging to 2 classes.
Found 76 images belonging to 2 classes.
Found 684 images belonging to 2 classes.
Found 76 images belonging to 2 classes.
Found 210 images belonging to 2 classes.


In [ ]:
import keras
from keras import layers

input_img = keras.Input(shape=(*image_resize, 3))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)


x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3),  padding='same', activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
encoder = keras.Model(input_img, encoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 400, 400, 3)]     0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 400, 400, 32)      896       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 200, 200, 32)      0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 200, 200, 64)      18496     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 100, 100, 64)      0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 100, 100, 128)     73856     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 50, 50, 128)       0   

In [ ]:
autoencoder.fit(
    train_generator_autoencoder,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator_autoencoder, 
    validation_steps = val_generator.samples // batch_size,
    epochs = 100, 
    verbose = 1)

Epoch 1/100
42/42 [==============================] - 38s 857ms/step - loss: 0.6144 - val_loss: 0.5342
Epoch 2/100
42/42 [==============================] - 35s 822ms/step - loss: 0.4906 - val_loss: 0.5260
Epoch 3/100
42/42 [==============================] - 34s 817ms/step - loss: 0.4818 - val_loss: 0.5285
Epoch 4/100
42/42 [==============================] - 34s 815ms/step - loss: 0.4806 - val_loss: 0.5275
Epoch 5/100
42/42 [==============================] - 35s 826ms/step - loss: 0.4708 - val_loss: 0.5234
Epoch 6/100
42/42 [==============================] - 35s 830ms/step - loss: 0.4792 - val_loss: 0.5234
Epoch 7/100
42/42 [==============================] - 35s 831ms/step - loss: 0.4695 - val_loss: 0.5224
Epoch 8/100
42/42 [==============================] - 35s 840ms/step - loss: 0.4693 - val_loss: 0.5244
Epoch 9/100
42/42 [==============================] - 36s 843ms/step - loss: 0.4753 - val_loss: 0.5228
Epoch 10/100
42/42 [==============================] - 35s 834ms/step - loss: 0.476

KeyboardInterrupt: ignored

In [ ]:
for l in encoder.layers:
    l.trainable = False
    print(l.name, l.trainable)

input_3 False
conv2d_14 False
max_pooling2d_6 False
conv2d_15 False
max_pooling2d_7 False
conv2d_16 False
max_pooling2d_8 False


In [ ]:
x = layers.Conv2D(64, (3, 3), strides = (2,2), padding='same', input_shape=[50,50,32])(encoded)

x = layers.ReLU()(x)
x = layers.MaxPooling2D((2,2))(x)

x = layers.Conv2D(128, (5, 5), strides = (2,2), padding='same')(x)

x = layers.ReLU()(x)
x = layers.MaxPooling2D((2,2))(x)

x = layers.Conv2D(64, (3, 3), strides = (2,2), padding='same')(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(1)(x)

model = keras.Model(input_img, x)

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 400, 400, 3)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 400, 400, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 200, 200, 16)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 200, 200, 8)       1160      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 100, 100, 8)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 100, 100, 8)       584       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 8)         0   

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics =["accuracy"])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator, 
    validation_steps = val_generator.samples // batch_size,
    epochs = 10, 
    verbose = 1)

Epoch 1/10
42/42 [==============================] - 26s 616ms/step - loss: 6.8713 - val_loss: 7.6246
Epoch 2/10
42/42 [==============================] - 25s 599ms/step - loss: 7.3505 - val_loss: 7.1481
Epoch 3/10
42/42 [==============================] - 25s 595ms/step - loss: 7.5504 - val_loss: 7.3863
Epoch 4/10
42/42 [==============================] - 25s 600ms/step - loss: 7.4409 - val_loss: 7.1481
Epoch 5/10
42/42 [==============================] - 25s 597ms/step - loss: 7.6011 - val_loss: 8.3394
Epoch 6/10
42/42 [==============================] - 25s 599ms/step - loss: 7.2739 - val_loss: 7.8629
Epoch 7/10
42/42 [==============================] - 25s 601ms/step - loss: 7.6765 - val_loss: 7.6246
Epoch 8/10
42/42 [==============================] - 25s 601ms/step - loss: 8.2357 - val_loss: 7.3863
Epoch 9/10
42/42 [==============================] - 25s 604ms/step - loss: 6.9795 - val_loss: 7.1481
Epoch 10/10
42/42 [==============================] - 25s 605ms/step - loss: 7.4777 - val_lo

In [ ]:
model.evaluate(test_generator, verbose = 1)

14/14 [==============================] - 7s 493ms/step - loss: 7.3223 - accuracy: 0.5198


[7.261542797088623, 0.523809552192688]